In [1]:
import plotly.graph_objects as go
import polars as pl
import pandas as pd
import numpy as np
import pandas_ta as ta
import sys
from yahoo_finance_api2 import share
from yahoo_finance_api2.exceptions import YahooFinanceError

In [2]:
my_share = share.Share('7203.T')
symbol_data = None
try:
    symbol_data = my_share.get_historical(
    share.PERIOD_TYPE_YEAR, 100,
    share.FREQUENCY_TYPE_DAY, 1)
except YahooFinanceError as e:
    print(e.message)
    sys.exit(1)
df = pd.DataFrame(symbol_data)
df["datetime"] = pd.to_datetime(df.timestamp, unit="ms")
df.head()

,timestamp,open,high,low,close,volume,datetime
0,925948800000,698.0,722.0,690.0,722.0,15575000,1999-05-06
1,926035200000,718.0,720.0,690.0,696.0,15165000,1999-05-07
2,926294400000,696.0,704.0,690.0,694.0,6305000,1999-05-10
3,926380800000,700.0,704.0,688.0,688.0,8430000,1999-05-11
4,926467200000,688.0,700.0,688.0,700.0,12980000,1999-05-12


In [3]:
import yfinance as yf
df2 = pd.DataFrame() # Empty DataFrame
df2 = df2.ta.ticker("aapl")

In [3]:
# VWAP requires the DataFrame index to be a DatetimeIndex.
# Replace "datetime" with the appropriate column from your DataFrame
df.set_index(pd.DatetimeIndex(df["datetime"]), inplace=True)

# Calculate Returns and append to the df DataFrame
df.ta.log_return(cumulative=True, append=True)
df.ta.percent_return(cumulative=True, append=True)

# New Columns with results
df.columns

# Take a peek
df.tail()

,timestamp,open,high,low,close,volume,datetime,CUMLOGRET_1,CUMPCTRET_1
datetime,,,,,,,,,
2023-06-07 00:00:00,1686096000000,2058.0,2078.5,2021.0,2022.0,41062900,2023-06-07 00:00:00,1.029817,1.800554
2023-06-08 00:00:00,1686182400000,2038.5,2052.0,2017.5,2024.0,28562300,2023-06-08 00:00:00,1.030806,1.803324
2023-06-09 00:00:00,1686268800000,2055.0,2057.5,2030.0,2051.5,37276500,2023-06-09 00:00:00,1.044301,1.841413
2023-06-12 00:00:00,1686528000000,2065.5,2073.5,2054.5,2069.0,19714600,2023-06-12 00:00:00,1.052796,1.865651
2023-06-13 06:15:01,1686636901000,2100.0,2183.0,2099.5,2173.5,70125200,2023-06-13 06:15:01,1.102069,2.010388


In [5]:
#plotly.graph_objectを使用してCUMLOGRETを折れ線で表示する
#plotly.graph_objectを使用してCUMLOGRETを折れ線で表示する
fig = go.Figure(data=go.Scatter(x=df.index, y=df['CUMLOGRET_1']))
#グラフにタイトルを追加する
fig.update_layout(title='7203.T')
#グラフに軸ラベルを追加する
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='CUMLOGRET')
#グラフを表示する
fig.show()

#plotly.graph_objectを使用してCUMLOGRETを折れ線で表示する

